In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
import random
import matplotlib.patheffects as PathEffects

In [0]:
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import os
import pickle
import matplotlib.pyplot as plt

In [0]:
from itertools import permutations
import seaborn as sns
from keras.datasets import mnist
from sklearn.manifold import TSNE
from sklearn.svm import SVC
import json


In [12]:
!unzip /content/drive/My\ Drive/drive-download-20191201T013728Z-001.zip

Archive:  /content/drive/My Drive/drive-download-20191201T013728Z-001.zip
replace testOutputPredictions.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
with open ('/content/triplets_dataset/behind_test_triplets.json') as json_file:
  behind_test=json.load(json_file)

with open ('/content/triplets_dataset/on_test_triplets.json') as json_file:
  on_test=json.load(json_file)


with open ('/content/triplets_dataset/under_test_triplets.json') as json_file:
  under_test=json.load(json_file)


with open ('/content/triplets_dataset/next to_test_triplets.json') as json_file:
  next_to_test=json.load(json_file)


In [0]:
def create_triplets(behind_train):

  new_dict={}
  for key in behind_train.keys():
    if key=="predicate":
      new_dict[key]=behind_train[key]
      continue
    ex=[]
    val=behind_train[key]
    val=list(val)
    total_length=len(val)
    print(total_length)
    for i in range(int(total_length/2)):
      a_p_n_pairs={}
      a_p_n_pairs["anchor"]=val[i]["filename1"]
      a_p_n_pairs["anchor_details"]=val[i]["positive"]
      a_p_n_pairs["positive"]=val[i+int(total_length/2)]["filename1"]
      a_p_n_pairs["positive_details"]=val[i+int(total_length/2)]["positive"]
      a_p_n_pairs["negative"]=val[i]["filename2"]
      a_p_n_pairs["negative_details"]=val[i]["negative"]
      ex.append(a_p_n_pairs)
      new_dict[key]=ex
  return new_dict
  

In [62]:
behind_test_triplets=create_triplets(behind_test)


177


In [63]:
print(behind_test_triplets)

{'predicate': 'behind', 'examples': [{'anchor': '5870797431_00a5f8a60d_b.jpg', 'anchor_details': {'bbox1': [102, 645, 185, 439], 'bbox2': [121, 602, 502, 779], 'obj1': 'person', 'obj2': 'person', 'relationship': 'behind'}, 'positive': '4126242775_1ffef299e8_o.jpg', 'positive_details': {'bbox1': [297, 772, 665, 1188], 'bbox2': [487, 635, 1166, 1263], 'obj1': 'truck', 'obj2': 'car', 'relationship': 'behind'}, 'negative': '7503712440_d88421f5db_b.jpg', 'negative_details': {'bbox1': [89, 222, 470, 680], 'bbox2': [34, 161, 688, 775], 'obj1': 'person', 'obj2': 'person', 'relationship': 'next to'}}, {'anchor': '161638840_4de67b36c9_o.jpg', 'anchor_details': {'bbox1': [135, 600, 336, 623], 'bbox2': [1, 171, 383, 488], 'obj1': 'person', 'obj2': 'chair', 'relationship': 'behind'}, 'positive': '5829524547_c86197d694_b.jpg', 'positive_details': {'bbox1': [332, 726, 479, 670], 'bbox2': [173, 352, 308, 735], 'obj1': 'person', 'obj2': 'person', 'relationship': 'behind'}, 'negative': '5297711298_16cf5

In [64]:
on_test_triplets=create_triplets(on_test)

366


In [65]:
under_test_triplets=create_triplets(under_test)

55


In [66]:
next_to_test_triplets=create_triplets(next_to_test)

325


In [30]:
!wget http://imagenet.stanford.edu/internal/jcjohns/scene_graphs/sg_dataset.zip

--2019-12-01 17:22:39--  http://imagenet.stanford.edu/internal/jcjohns/scene_graphs/sg_dataset.zip
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2000609775 (1.9G) [application/zip]
Saving to: ‘sg_dataset.zip’

sg_dataset.zip      100%[===================>]   1.86G  48.8MB/s    in 58s     

2019-12-01 17:23:37 (32.8 MB/s) - ‘sg_dataset.zip’ saved [2000609775/2000609775]



In [31]:
!unzip sg_dataset.zip

Archive:  sg_dataset.zip
   creating: sg_dataset/
  inflating: sg_dataset/sg_test_annotations.json  
   creating: sg_dataset/sg_test_images/
  inflating: sg_dataset/sg_test_images/10002158834_6dd6c6667f_b.jpg  
  inflating: sg_dataset/sg_test_images/10003466765_d20a7655c6_b.jpg  
  inflating: sg_dataset/sg_test_images/10008170616_a54deca9e3_b.jpg  
  inflating: sg_dataset/sg_test_images/10050248663_2cdb49c115_b.jpg  
  inflating: sg_dataset/sg_test_images/1006083276_0c1a4345fb_o.jpg  
  inflating: sg_dataset/sg_test_images/10085008474_8d72a9dc5e_b.jpg  
  inflating: sg_dataset/sg_test_images/10104268563_965a646010_b.jpg  
  inflating: sg_dataset/sg_test_images/10132007626_6421a783be_b.jpg  
  inflating: sg_dataset/sg_test_images/10132084253_cfa22219a1_b.jpg  
  inflating: sg_dataset/sg_test_images/10142659205_1158853119_b.jpg  
  inflating: sg_dataset/sg_test_images/10142737776_a40de33ab1_b.jpg  
  inflating: sg_dataset/sg_test_images/10161340284_8a72090a5b_b.jpg  
  inflating: sg_data

In [0]:
import imageio
from PIL import Image

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [0]:
def bbox_mask(img_filename,details):
  try:
    img1 = imageio.imread('/content/sg_dataset/sg_test_images/'+img_filename)
  except:
    return "null"
    
  img1 = np.array(img1)
  img1 = img1*0.0
  sub_box=details["bbox1"]
  ob_box=details["bbox2"]
  bboxes = [sub_box[0], sub_box[1], sub_box[2], sub_box[3], ob_box[0], ob_box[1], ob_box[2], ob_box[3]]
  img1[bboxes[0]:bboxes[1], bboxes[2]:bboxes[3]] = 0.5    
  img1[bboxes[4]:bboxes[5], bboxes[6]:bboxes[7]] += 1.0
  img1[bboxes[4]:bboxes[5], bboxes[6]:bboxes[7]] /= 2.0
  img1[bboxes[4]:bboxes[5], bboxes[6]:bboxes[7]] /= 0.75
  img1 = rgb2gray(img1)
  img1 = np.array((img1 * 255).astype(np.uint8))
  img1 = Image.fromarray(img1)
  img1 = img1.resize((100, 100), Image.ANTIALIAS)

  img1 = np.array(img1)
  img1 = img1/255.0
  
  img1 = np.reshape(img1, newshape=[1,img1.shape[0],img1.shape[1],1])

  return img1[0][:,:,0]

In [69]:
import matplotlib.pyplot as plt

X=[]
y=[]
t=0
for i in behind_test_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  if(anchor=="null"):
    continue
  positive=bbox_mask(i["positive"],i["positive_details"])
  if(positive=="null"):
    continue
  negative=bbox_mask(i["negative"],i["negative_details"])
  if(negative=="null"):
    continue
  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
  trip=[anchor_flat,positive_flat,negative_flat]
  X.append(trip)
  y.append(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 35520512 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.

In [71]:
for i in on_test_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  positive=bbox_mask(i["positive"],i["positive_details"])
  negative=bbox_mask(i["negative"],i["negative_details"])
  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
 
  trip=[anchor_flat,positive_flat,negative_flat]
  #trip=np.asarray(trip)
  X.append(trip)
  y.append(0)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 35520512 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20054016 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 34865152 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))


In [0]:
for i in under_test_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  positive=bbox_mask(i["positive"],i["positive_details"])
  negative=bbox_mask(i["negative"],i["negative_details"])
  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
 
  trip=[anchor_flat,positive_flat,negative_flat]
  #trip=np.asarray(trip)
  X.append(trip)
  y.append(3)
  

In [73]:
for i in next_to_test_triplets["examples"]:

  trip=[]
  anchor=bbox_mask(i["anchor"],i["anchor_details"])
  positive=bbox_mask(i["positive"],i["positive_details"])
  negative=bbox_mask(i["negative"],i["negative_details"])
  anchor_flat=anchor.reshape(-1)
  positive_flat=positive.reshape(-1)
  negative_flat=negative.reshape(-1)
 
  trip=[anchor_flat,positive_flat,negative_flat]
  #trip=np.asarray(trip)
  X.append(trip)
  y.append(1)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 35520512 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20054016 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 34865152 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))


In [74]:
X_test=np.asarray(X)
y_test=np.asarray(y)
print(X_test.shape)
print(y_test.shape)

(458, 3, 10000)
(458,)


In [0]:
import random
p=np.arange(X_test.shape[0])
random.shuffle(p)
X_test_new=X_test[p,:]
y_test_new=y_test[p]

In [76]:
print(X_test_new.shape)
print(y_test_new.shape)

(458, 3, 10000)
(458,)


In [0]:
np.save('/content/X_test.npy',X_test_new)
np.save('/content/y_test.npy',y_test_new)

In [0]:
np.save('/content/drive/My Drive/X_test.npy',X_test_new)
np.save('/content/drive/My Drive/y_test.npy',y_test_new)